# Performance to Sound Object Processing

The purpose of this script is to break down all the metatone touch-data into individual sound object CSVs. Each performance is split per player, and into 5-second windows at 1-second intervals. The output CSVs have a `time` column with the number of seconds since the start of the file and also have a `moving` column which is 0 if velocity is 0 and is otherwise 1.

In [42]:
%matplotlib inline
from __future__ import print_function
import os
import time
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import seaborn as sns
import random

In [5]:
log_files = []
performances = []
for local_file in os.listdir("data"):
    if local_file.endswith("-touches.csv"):
        log_files.append("data/" + local_file)
# for log in log_files:
    # do some stuff
    
log_files

['data/2013-04-20T14-55-00-MetatoneOSCLog-touches.csv',
 'data/2013-04-21T11-38-00-MetatoneOSCLog-touches.csv',
 'data/2013-04-27T17-29-00-MetatoneOSCLog-touches.csv',
 'data/2013-05-04T12-12-00-MetatoneOSCLog-touches.csv',
 'data/2013-05-04T14-23-00-MetatoneOSCLog-touches.csv',
 'data/2013-07-01T17-20-46-MetatoneOSCLog-touches.csv',
 'data/2013-08-03T17-10-17-MetatoneOSCLog-touches.csv',
 'data/2013-08-03T17-25-19-MetatoneOSCLog-touches.csv',
 'data/2013-08-03T18-37-03-MetatoneOSCLog-touches.csv',
 'data/2013-08-03T18-49-35-MetatoneOSCLog-touches.csv',
 'data/2014-03-08T11-49-15-MetatoneOSCLog-touches.csv',
 'data/2014-03-08T12-14-57-MetatoneOSCLog-touches.csv',
 'data/2014-03-08T12-21-29-MetatoneOSCLog-touches.csv',
 'data/2014-03-13T15-28-14-MetatoneOSCLog-touches.csv',
 'data/2014-03-13T16-11-33-MetatoneOSCLog-touches.csv',
 'data/2014-03-15T12-33-18-MetatoneOSCLog-touches.csv',
 'data/2014-03-15T12-55-15-MetatoneOSCLog-touches.csv',
 'data/2014-03-15T13-19-15-MetatoneOSCLog-touche

In [92]:
##

log_frames = []

print("Loading all the frames.")
for log in log_files:
    log_frames.append(pd.DataFrame.from_csv(log,parse_dates=True,header=0))
print("Concatenating all the frames.")
all_logs_frame = pd.concat(log_frames)
print("Done.")

Loading all the frames.
Concatenating all the frames.
Done.


In [108]:



#all_logs_frame.describe()
# print(all_logs_frame.describe())
#pd.concat(log_frames)

print("Generating the 1s time index")
# a big dataframe of each second throughout the whole period.
times = pd.DataFrame(all_logs_frame.index,index=all_logs_frame.index).resample('1s', how='count')
#sound_object_frames = pd.DataFrame(all_logs_frame.index,index=all_logs_frame.index)
print("Generating all the sound object frames")
times["sound_object_frames"] = times.apply(five_second_frame, axis=1, frame=all_logs_frame)
#times = times.dropna()


Generating the 1s time index


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:18: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()


Generating all the sound object frames


KeyboardInterrupt: 

In [109]:
## Load all touch files

a = log_files[0]

def five_second_frame(row, frame):
    """
    Returns a dataframe of five seconds length before "time".
    """
    end_time = row.name
    frame = frame.between_time((end_time + timedelta(seconds=-5)).time(), end_time.time())
    if frame.empty:
        return np.nan
    return frame

def clean_sound_object(frame):
    """Cleans up sound object frames by removing unneeded 
        columns and changing times to differences."""
    first_time = frame.index[0].to_datetime()
    output = frame[['x_pos','y_pos','velocity']]
    output['t'] = output.index
    output.t = (output.t - first_time).apply(timedelta.total_seconds)
    
    return output


    
def split_and_classify_metatone_log(log_frame):
    return [],[]

log_frame = pd.DataFrame.from_csv(a,parse_dates=True,header=0)
log_frame["moving"] = log_frame.velocity/log_frame.velocity
log_frame.moving = log_frame.moving.fillna(0)
times = pd.DataFrame(log_frame.index,index=log_frame.index).resample('1s', how='count')
sound_object_frames = pd.DataFrame(log_frame.index,index=log_frame.index)
times["sound_object_frames"] = times.apply(five_second_frame, axis=1, frame=log_frame)

#for performer in log_frame.device_id.unique():
#    sound_object_frames[performer] = times.apply(five_second_frame, axis=1, frame=log_frame, name=performer)

    

#out = times.apply(five_second_frame, axis=1, frame=log_frame, name='charles')

# 1. Split via performer
# 2. Split into rolling five second segments (should be a list)
# 3. Produce a feature vector and classify gesture for each segment (record separately)
# 4. Calculate time and moving columns
# 5. Save each non-empty segment into a CSV file.



/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:33: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()


In [86]:
df = times.sound_object_frames.iloc[57].copy()
clean_sound_object(df)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: to_datetime is deprecated. Use self.to_pydatetime()
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,x_pos,y_pos,velocity,t
time,,,,
2013-04-20 14:59:44.002254,862.0,92.0,1.118034,0.000000
2013-04-20 14:59:44.015968,862.0,93.5,1.500000,0.013714
2013-04-20 14:59:44.039583,861.5,94.5,1.118034,0.037329
2013-04-20 14:59:44.050579,861.0,96.0,1.581139,0.048325
2013-04-20 14:59:44.068206,861.0,97.0,1.000000,0.065952
2013-04-20 14:59:44.086116,860.5,98.5,1.581139,0.083862
2013-04-20 14:59:44.104501,860.0,99.5,1.118034,0.102247
2013-04-20 14:59:44.120240,860.0,101.0,1.500000,0.117986
2013-04-20 14:59:44.135594,860.0,101.5,0.500000,0.133340
